## Overall Prediction Observation


In [16]:
# Define variables
IS_FILTERED = True
ARCHIVED = None

SHOW_HEADER = True
SHOW_LAST_APPEARANCE = None

In [17]:
# Import libraries
import sys
sys.path.append('../../../prediction')

import numpy as np
import matplotlib.pyplot as plt

from putils.observation import compute_rmse, compute_mae, compute_mape, load_data_from_tuned_folder

In [18]:
# Load data
dfs = {
    "l1": load_data_from_tuned_folder("l1", last_appearance=SHOW_LAST_APPEARANCE),
    "l2": load_data_from_tuned_folder("l2", last_appearance=SHOW_LAST_APPEARANCE),
    "l3": load_data_from_tuned_folder("l3", last_appearance=SHOW_LAST_APPEARANCE),
}

# Make L1 and L2 data have the same length with L3 if SHOW_LAST_APPEARANCE = None
if SHOW_LAST_APPEARANCE is None:
    dfs["l1"] = dfs["l1"].tail(len(dfs["l3"]))
    dfs["l2"] = dfs["l2"].tail(len(dfs["l3"]))

# Indicate which columns to ignore
IGNORED_COLS = ["Time", "Actual"] # No need to include "FormattedTime" because it is used as index
if IS_FILTERED: IGNORED_COLS.append("Raw")

In [19]:
# Compute RMSE, MAE, and MAPE for each model
for key, df in dfs.items():
    print("Layer", key)
    for column in df.columns:
        if column not in IGNORED_COLS:
            actual = df["Raw"] if IS_FILTERED else df["Actual"]
            rmse = compute_rmse(actual, df[column])
            mape = compute_mape(actual, df[column])
            mae = compute_mae(actual, df[column])
            print(f"{column} - RMSE: {rmse:.6f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%")
    print("------------------")

Layer l1
ARIMA - RMSE: 0.022843, MAE: 0.0153, MAPE: 2.80%
RNN - RMSE: 0.014873, MAE: 0.0103, MAPE: 1.88%
LSTM - RMSE: 0.015537, MAE: 0.0106, MAPE: 1.94%
CNN - RMSE: 0.015870, MAE: 0.0108, MAPE: 1.97%
GRU - RMSE: 0.015183, MAE: 0.0105, MAPE: 1.92%
------------------
Layer l2
LINEAR_REGRESSION - RMSE: 0.014865, MAE: 0.0103, MAPE: 1.87%
RANDOM_FOREST - RMSE: 0.015251, MAE: 0.0105, MAPE: 1.90%
FEEDFORWARD_NEURAL_NETWORK - RMSE: 0.014886, MAE: 0.0103, MAPE: 1.87%
------------------
Layer l3
Predicted - RMSE: 0.014942, MAE: 0.0103, MAPE: 1.87%
------------------


In [20]:
l2_predictions = dfs["l2"].drop(columns=IGNORED_COLS)
average_predictions = l2_predictions.mean(axis=1)

rmse = compute_rmse(dfs["l2"]["Raw"], average_predictions)
print(f"Average RMSE: {rmse:.6f}")

Average RMSE: 0.014943
